# Распознавание языка текста

<hr>

С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Статический текст](#Статический-текст)
- [Динамический текст](#Динамический-текст)
    - [Построение модели](#Построение-модели)
    - [Проверка динамического распознавания](#Проверка-динамического-распознавания)

Подключение библиотек:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
import sys
sys.path.insert(0, "../lib/")
from datasets import fetch_20languages

## Статический текст

[Набор данных](https://huggingface.co/datasets/papluca/language-identification)

In [ ]:
# Загрузка данных
dataset = fetch_20languages(return_X_y=True)

# Вывод описания
print(dataset.DESCR)

In [ ]:
df = dataset.data['train']
df.head()

In [ ]:
# df = pd.read_csv('../data/lang_detector/train.csv')
# df.head()

In [ ]:
df.describe()

In [ ]:
# Количество текстов по каждому классу
df['labels'].value_counts()

In [ ]:
# Среднее количество символов в текстах по каждому классу
df.groupby('labels').agg(
    lambda group: group['text'].str.len().mean()
)

In [ ]:
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

In [ ]:
pipeline.fit(df['text'], df['labels'])

In [ ]:
df_test = pd.read_csv('../data/lang_detector/test.csv')

In [ ]:
pipeline.score(df_test['text'], df_test['labels'])

## Динамический текст

### Построение модели

In [ ]:
import re

In [ ]:
# Исходные данные
df.head()

In [ ]:
# Шаблон для делителя строки на слова
COMPILER = re.compile("\W+", re.UNICODE)


def split_sentence(lang, text):
    s = list()
    for word in set(COMPILER.split(text)):
        if word:
            s.append((lang, word))
    return s


# Формирование списка пар язык-слово
data = list()
for i, row in df.iterrows():
    data += split_sentence(row['labels'], row['text'])
    
    
data[:5]

In [ ]:
# Формивание датафрейма язык-слово и удаление повторений
df_new = pd.DataFrame(data=data, columns=['labels', 'word']).drop_duplicates()
df_new.head()

In [ ]:
INPUT = 'обуч'

print(
    df_new[df_new['word'].str.contains(INPUT)]\
        .groupby('labels')\
        .count().T
)

In [ ]:
# Априорные вероятности классов
# class_prior=[
#     0.04, 0.04, 0.05, 0.05, 0.1, 0.05, 0.05, 0.04, 0.05, 0.05,
#     0.05, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.05, 0.05, 0.05
# ]
class_prior=[0.05]*20

# Пострение модели классификации
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char', ngram_range=(2,4))),
    ('classifier', MultinomialNB(class_prior=class_prior))
])

# Обучение модели
pipeline.fit(df_new['word'], df_new['labels'])

print(
    # Оценка качества на тестовом множестве (из первой задачи)
    f"Accuracy = { pipeline.score(df_test['text'], df_test['labels']) }"
)

In [ ]:
# pipeline.named_steps['vectorizer'].vocabulary_

In [ ]:
# Классы
langs = pipeline.named_steps['classifier'].classes_
langs

In [ ]:
INPUT = 'обуч'

# Вероятности принадлежности классам для некоторого слова
probs = pipeline.predict_proba([INPUT,])[0]
probs

### Проверка динамического распознавания 

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
def display_prediction(langs, probs):
    """
    Отображение вероятностей по языкам 
    в виде датафрейма.
    """
    print(
        pd.DataFrame(
            data=zip(langs, probs),
            columns=['lang', 'prob']
        )\
        .sort_values('prob', ascending=0)\
        .head(10)
    )

In [ ]:
# Ввод текста
text_input = widgets.Text()
display(text_input)

# Вывод результата предсказания
output = widgets.Output()
display(output)


def handle_process_text(sender):
    with output:
        clear_output()
        probs = pipeline.predict_proba([sender.new,])[0]
        langs = pipeline.named_steps['classifier'].classes_
        display_prediction(langs, probs)


# Отслеживание ввода
text_input.observe(handle_process_text, names='value')